[View in Colaboratory](https://colab.research.google.com/github/restrepo/medicion/blob/master/cienciometria/index.ipynb)

# Combinación de bases de datos de artículos científicos
```
URL_base=https://colab.research.google.com/github/blob_file_in_repo
blob_file_in_repo → restrepo/medicion/blob/master/cienciometria/merge.ipynb

```
## UdeA
### Combinar bases de datos bibliográficas UDEA
* [Combibnar W0S+SCP+SCI](https://colab.research.google.com/github/restrepo/medicion/blob/master/cienciometria/merge.ipynb) versión detallada
* [Combibnar W0S+SCP+SCI](https://colab.research.google.com/github/restrepo/medicion/blob/master/cienciometria/WOS_SCI_SCP.ipynb) versión corta
##  [Open Access](https://colab.research.google.com/github/restrepo/medicion/blob/master/cienciometria/OpenAccess.ipynb) DOAJ API 
##  [Open Access](https://colab.research.google.com/github/restrepo/medicion/blob/master/cienciometria/XXX.ipynb)  

https://unpaywall.org/ dataset. See https://towardsdatascience.com/interactively-analyse-100gb-of-json-data-with-spark-e018f9436e76
```bash
# Select True Open Access articles
grep -E '"is_oa":\s*true' unpaywall_snapshot_2018-06-21T164548_with_versions.jsonl  >  oa.json
# Select only DOIS values: for True Opena Access articles
echo DOI > oadoi.csv
awk -F","  '{print $1}' oa.json | grep '"doi":' | awk -F '"' '{print $(NF-1)}'   >> oadoi.csv
# Full DOIs
echo DOI > dois.csv
awk -F","  '{print $1}' unpaywall_snapshot_2018-06-21T164548_with_versions.jsonl | grep '"doi":' | awk -F '"' '{print $(NF-1)}'   >> dois.csv
```
Results here:
1. [oadois.csv](https://drive.google.com/file/d/1uipr7JlNhLzZK5W8VymUNwVGGim-AqhC/view?usp=sharing)
1. [dois.csv]()
* TODO: comprobar los APIS de http://blog.impactstory.org/introducting-oadoi/. Por ejemplo
Usar
* http://oadoi.org/10.1103/PhysRevD.98.055016
En lugar de   http:://doi.org/10.1103/PhysRevD.98.055016

In [0]:
import pandas as pd

Gray Open Acces format

In [7]:
grayoa=pd.DataFrame()
grayoa.append( {"doi": "10.1103/physrevd.86.075011", "year": 2012, "genre": "journal-article", "is_oa": True, 
              "title": "Connection of gamma rays, dark matter, and Higgs boson searches at the LHC",
              "doi_url": "https://doi.org/10.1103/physrevd.86.075011", "updated": "2018-06-18T03:46:10.784962",
              "publisher": "American Physical Society (APS)", "z_authors": [{"given": "J. D.", "family": "Ruiz-Alvarez"}, 
                                                                            {"given": "C. A.", "family": "de S. Pires"}, 
                                                                            {"given": "Farinaldo S.", "family": "Queiroz"}, 
                                                                            {"given": "D.", "family": "Restrepo"}, 
                                                                            {"given": "P. S.", "family": "Rodrigues da Silva"}], 
              "journal_name": "Physical Review D", "oa_locations": [{"url": "http://arxiv.org/pdf/1206.5779", "pmh_id": "oai:arXiv.org:1206.5779",
                                                                     "is_best": True, "license": 'null', "updated": "2018-06-18T03:46:10.783600", 
                                                                     "version": "submittedVersion", "evidence": "oa repository (via OAI-PMH doi match)", 
                                                                     "host_type": "repository", "url_for_pdf": "http://arxiv.org/pdf/1206.5779",
                                                                     "url_for_landing_page": "http://arxiv.org/abs/1206.5779"}], 
              "data_standard": 2, "journal_is_oa": False, "journal_issns": "1550-7998,1550-2368", "published_date": "2012-10-08", 
              "best_oa_location": {"url": "http://arxiv.org/pdf/1206.5779", "pmh_id": "oai:arXiv.org:1206.5779", "is_best": True, "license": 'null',
                                   "updated": "2018-06-18T03:46:10.783600", "version": "submittedVersion", 
                                   "evidence": "oa repository (via OAI-PMH doi match)", "host_type": "repository", 
                                   "url_for_pdf": "http://arxiv.org/pdf/1206.5779", "url_for_landing_page": "http://arxiv.org/abs/1206.5779"}, 
              "journal_is_in_doaj": False},ignore_index=True)


,best_oa_location,data_standard,doi,doi_url,genre,is_oa,journal_is_in_doaj,journal_is_oa,journal_issns,journal_name,oa_locations,published_date,publisher,title,updated,year,z_authors
0,"{u'updated': u'2018-06-18T03:46:10.783600', u'...",2.0,10.1103/physrevd.86.075011,https://doi.org/10.1103/physrevd.86.075011,journal-article,1.0,0.0,0.0,"1550-7998,1550-2368",Physical Review D,"[{u'updated': u'2018-06-18T03:46:10.783600', u...",2012-10-08,American Physical Society (APS),"Connection of gamma rays, dark matter, and Hig...",2018-06-18T03:46:10.784962,2012.0,"[{u'given': u'J. D.', u'family': u'Ruiz-Alvare..."
